In [ ]:
## tidied up scripts -- Mar 2017

#### NOTE:

Previousely used NYT's CRF ingredient tagger as a means to 'clean' up the ingredients a little more...   
** This script adds additional functions to improve the structuring **   

Refs.  
- https://github.com/hengrumay/ingredient-phrase-tagger  || https://github.com/NYTimes/ingredient-phrase-tagger  
- http://open.blogs.nytimes.com/2015/04/09/extracting-structured-data-from-recipes-using-conditional-random-fields/?_r=1
- https://rajmak.wordpress.com/2016/02/19/structuring-text-using-conditional-random-field-crf-tagging-recipe-ingredient-phrases/  

-- Previous script: `Re-coding_parseIngredients_run_CRFtggerNsomecleanup.ipynb`  
-- Ref prior code exploration: `Retrieve_CRFtaggedCleanedFilesNtidy4use_v0.ipynb`

####   

In [ ]:
from __future__ import print_function, division

import pandas as pd
import numpy as np

import pickle

## --------------------------------------------
import re
#==============================================

from __future__ import print_function

import sys
import os
import tempfile

from ingredient_phrase_tagger.training import utils

import sklearn
import pycrfsuite

In [ ]:
filepath = '/Users/hrm/Documents/Dropbox/DSrelated/Metis/recipes/bbc_goodfood_recipes_pickles/'
DF = pd.read_pickle(filepath+'BBCgoodfood_10081recipesNlinks.pkl')

In [ ]:
## MUST change dir to run following scripts! Do Once...
# os.chdir(os.path.dirname('./ingredient-phrase-tagger-master/'))
# !pwd
# /Users/hrm/Documents/Dropbox/DSrelated/Metis/recipes/ingredient-phrase-tagger-master

# print(os.getcwd())
# os.getcwd() 

In [ ]:
Modelpath = '/Users/hrm/Documents/Dropbox/DSrelated/Metis/recipes/ingredient-phrase-tagger-master/'
Folderpath = '/Users/hrm/Documents/Dropbox/DSrelated/Metis/recipes'

crf_filespath = Folderpath +'/crf_tagger_files/'

# list(DF.index)

# filename = [crf_filespath + 'bbcgdfd_'+str(i)+'.txt' for i in list(DF.index)]
filename = [crf_filespath + 'bbcgdfd_'+str(i)+'.txt' for i in range(10081)]
# filename

In [ ]:
def getCAPcharIndices(s):
    return [i for i, c in enumerate(s) if c.isupper()]

def getBracketIndices(s):
    return [i for i, c in enumerate(s) if c=='(']

def getcommaIndices(s):
    return [i for i, c in enumerate(s) if c==',']

def getdashIndices(s):
    return [i for i, c in enumerate(s) if c=='-']

In [ ]:
unitlabels = ['tablespoon', 'teaspoon', 'liter', 'milliliter', 'ml', 'kilogram', 'gram', 
              'g,','kg','cm','mm','glass', 'pint', 'strips',
              'splash' ,'slice','handful','sprig','sprigs','clove','cloves', 'thumb-size piece',
             'pinch','knob'] #

def checkUnitLabels(s):
    for u in unitlabels:
        #s = s.replace(u, ' '+ u + ' ')
        s = re.sub(r'(?<=[\d\s*])%s()'%(u), ' '+ u + ' ', s)
        #s = s.replace('glass',' glass ')
    return s

In [ ]:
## check if str can be converted to float...
# http://edumaven.com/python-programming/is-number 

def is_float(input):
    try:
        num = float(input)
    except ValueError:
        return False
    return True

def is_int(input):
    try:
        num = int(input)
    except ValueError:
        return False
    return True

In [ ]:
# restructure other info

def getUnitsInOther(df0):
    
    o = [] #np.empty(shape=(len(df0),1))
    if 'other' in df0.columns.tolist():
        for n in range(len(df0.other)):
            try:
                tmp = checkUnitLabels(df0.other[n])
                tmp = tmp.replace('*',' * ')
                tmp = tmp.replace('+',' + ')
                tmp = tmp.replace('of','')
                tmp = tmp.replace('-',' : ')

                #print(tmp)

                #if any tmp is in unitlabels -- then append?
                t = [tmp for u in unitlabels if u in tmp and '/' not in tmp ]                
                
                if t!=[] and ')' not in tmp and '(' not in tmp:
                    o.append(tmp)
                else:
                    o.append('')
                #print(n, tmp)

            except:
                #error_id.append(n)
                o.append('')
                #print(n, error_id)
    else:
        o = ['']*len(df0)

    return o 

In [ ]:
unitlabels
# ['tablespoon',
#  'teaspoon',
#  'liter',
#  'milliliter',
#  'ml',
#  'kilogram',
#  'gram',
#  'g,',
#  'kg',
#  'cm',
#  'mm',
#  'glass',
#  'pint',
#  'strips',
#  'splash',
#  'slice',
#  'handful',
#  'sprig',
#  'sprigs',
#  'clove',
#  'cloves',
#  'thumb-size piece',
#  'pinch',
#  'knob']

In [ ]:
# restructure qty info

def getUnitsInQty(df0):
    #error_id=[]
    q = [] #np.empty(shape=(len(df0),1))
    for n in range(len(df0.qty)):
        try:
            
#             for u in unitlabels: 
#                 if u in df0.name[n]:
#                     tmp =             
#             if pd.isnull(df0.unit[n]) 
            
            tmp = checkUnitLabels(df0.qty[n])
            tmp = tmp.replace('*',' * ')
            tmp = tmp.replace('+',' + ')
            tmp = tmp.replace('g-',' gram-')
            tmp = re.sub(r'(?<=[\d\s*])%s()'%('g-'), ' gram-', tmp)
            
            if 'sundried-tomato' or 'thumb-size' or 'oven-ready' or 'hot-smoked' not in tmp:
                tmp = tmp.replace('-',' : ')
            else:
                for w in ['oven-ready', 'hot-smoked']:
                    tmp = tmp.replace('w','')
                    df0.comment[n].extend(w) 
            
            tmp = tmp.replace('to',' : ')
            tmp = tmp.replace('a ','')
            

            #print(tmp)
            q.append(tmp)

            #t=[tmp for u in unitlabels if u in tmp]
            #if t!=[]:
            #    q.append(tmp)
            #else:
            #    q.append('')
            #print(n, tmp)

        except:
            #error_id.append(n)
            q.append('')
            #print(n, error_id)

    return q #, len(q)    

In [ ]:
def combine_qNo(q,o):
    qo = []
    for n in range(len(q)):
        qo.append((q[n] + o[n]).replace(',',''))#.replace('-','--'))

    return qo


In [ ]:
def getmisplacedUnQnS(qo):
    misplacedU=[]
    misplacedQnS =[]


    for n in range(len(qo)):
        #misplacedU.extend([(n,idx,w) for idx,w in enumerate(q[n].split(' ')) if w in unitlabels])
        #misplacedN.extend([(n,idx,w) for idx,w in enumerate(q[n].split(' ')) if is_float(w)])
        #if q[n] != np.nan:

        tmp_f=[]
        for idx,w in enumerate(qo[n].split(' ')):
            if w in unitlabels:
                misplacedU.append([n,idx,w])

            if is_float(w) or w in ['*','+', ':']:
                #print(w)
                tmp_f.append(w)

        misplacedQnS.append(tmp_f)

    return misplacedU, misplacedQnS

In [ ]:
# CHECKS and Replace misplaced unit in unit1
def checksNupdatesUnit(df0):
    
    for n,idx,w in misplacedU:
        df0.unit1[n] = w 

    if misplacedU == []:
        for n in range(len(df0)):
            if pd.isnull(df0.qty[n])==True and pd.isnull(df0.unit[n])==True and df0.comment[n]=='1 each of':
                df0.unit1[n] = 'each'
                df0.qty1[n] = 1
    return df0

In [ ]:
## Replace QTY
# checks and evaluates strings--floats
def checkNevalQty(misplacedQnS,df0,qo):

    for n in range(len(misplacedQnS)):
        w = misplacedQnS[n]
        #print(w)
        if w !=[]:
            
            if '*:' in ''.join(w):
                df0.qty1[n] = eval(''.join(''.join(w).split('*:')))
            
            elif len(w)==1 and w==[':']:
                df0.qty1[n] = np.nan
            
            elif len(w)==2 and w[-1]==':':
                df0.qty1[n] = eval(w[0])
            
            #elif w[0]==':':
            #    df0.qty1[n] = eval(w[1:])
            
            elif len(w)>2 and w[-1]==':':
                df0.qty1[n] = eval(' '.join(w[:-1]))
            
            
            elif 3>=len(w)>1 and w[1]==':':
                df0.qty1[n] = float(np.mean([float(i) for i in ' '.join(w).split(':')]))

            elif 5>=len(w)>3 and w[1]==':':
                df0.qty1[n] = float(np.mean([eval(i) for i in  ' '.join(w).split(':')]))
                
            elif 5>=len(w)>3 and w[3]==':':
                df0.qty1[n] = float(np.mean([eval(i) for i in  ' '.join(w).split(':')]))
            
            elif len(w)==5 and w[1]==':':
                if '*:' in ''.join(w):
                    df0.qty1[n] = eval(''.join(''.join(w[:-1]).split('+:')))
            
            elif len(w)==4 and ''.join(w[1:3])=='*:':
                df0.qty1[n] = float(np.mean([eval(i) for i in  ' '.join(w).split('* :')]))
                
            elif len(w)>5 and w[1]==':' and w[-1] ==':':
                df0.qty1[n] = float(np.mean([eval(i) for i in  ' '.join(w[:-1]).split(':')]))
                

            
            elif len(w)==3 and ''.join(w[-2:])=='*:':
                df0.qty1[n] = eval(' '.join(w[:-2]))

            elif w[-1] !='*':
                df0.qty1[n] = eval(' '.join(w))

            elif w[-1] =='*' and pd.isnull(df0.name[n])==True:
                df0.qty1[n] = eval(' '.join(w + ['1']))
                df0.name1[n] = qo[n].split()[-1]

            elif w[-1] =='*' and pd.isnull(df0.name[n])==False:
                df0.qty1[n] = eval(' '.join(w + ['1']))
                
                

        # if qty1/unit1 == empty or nan --> check if input contains 'to serve' --> name1
        if pd.isnull(df0.unit1[n])==True and pd.isnull(df0.name[n])==True: #== np.nan:
#             if 'to serve' in df0.input[n]:
#                 df0.name1[n] = df0.input[n][:df0.input[n].find('to serve')] + 'to serve'
#             if 'to dust' in df0.input[n]:
#                 df0.name1[n] = df0.input[n][:df0.input[n].find('to dust')] + 'to dust'
#             if 'for dusting' in df0.input[n]:
#                  df0.name1[n] = df0.input[n][:df0.input[n].find('for dusting')] + 'for dusting'
#             if 'for decorating' in df0.input[n]:
#                  df0.name1[n] = df0.input[n][:df0.input[n].find('for decorating')] + 'for decorating'

            if 'for' in df0.input[n]:
                df0.name1[n] = np.nan
                
            if 'to' in df0.input[n]:
                df0.name1[n] = np.nan 
            
            if 'and' in df0.input[n]:
                df0.name1[n] = np.nan
                
            
            #elif pd.isnull(df0.qty[n]):

        if pd.isnull(df0.name[n])==True and pd.isnull(df0.name1[n])==True and pd.isnull(df0.comment[n])==False:
            commaID = df0.comment[n].find(',')
            capID = getCAPcharIndices(df0.comment[n])

            if commaID!=[] and capID!=[]:
                if capID[0]>len(df0.comment[n])-commaID:
                    df0.name1[n] = df0.comment[n][:commaID] #'#'#df0.unit[n]
                elif capID[0]<len(df0.comment[n])-commaID:
                    df0.name1[n] = df0.comment[n][:capID[0]] #'#'#df0.unit[n]
                    df0.comment[n] = df0.comment[n][:capID[0]]
            else:
                df0.name1[n] = df0.comment[n]
        
        if pd.isnull(df0.name[n])==True and pd.isnull(df0.name1[n])==False:
            df0.name1[n] = df0.name[n]

        #elif pd.isnull(df0.unit[n])==False and df0.unit[n] not in unitlabels:
        #    df0.name1[n] = df0.unit[n] + ' ' + df0.name[n]

        #if pd.isnull(df0.unit1[n])==False:
        #    if '--' in df0.qty[n]:
    #   #      print('found')
        #        df0.qty1[n] = df0.qty[n][0] #np.mean([int(i) for i in ' '.join(w).split('--')])

    return df0

In [ ]:
def checkIngdname(s):
    #list(set([i for i in df0.name1[n].lower().split(' ') if df0.name1[n].lower().split(' ').count(i)>1]))[0]
    #list(pd.value_counts(df0.name1[3].lower().split(' ')).index)
    #list(pd.value_counts(df0.name1[3].lower().split(' ')))

    repeatedWds = list(set([i for i in s.lower().split(' ') if s.lower().split(' ').count(i)>1]))#[0]
    return repeatedWds

In [ ]:
# CheckNChangeRepeatedIngName
def CheckNChangeRepeatedIngName(df0):
    
    for n in range(len(df0.name1)):
        if pd.isnull(df0.name1[n])==False:
            
            if checkIngdname(df0.name1[n]) ==[]: 
                if pd.isnull(df0.comment[n])==False and checkIngdname(df0.comment[n]) ==[]:
                    continue
        #         elif checkIngdname(df0['input'][n]) ==[]:
        #             df0.name1[n] = 

            elif checkIngdname(df0.name1[n]) !=[] or checkIngdname(df0['input'][n]) !=[]: 
                if checkIngdname(df0.name1[n]) !=[] and len(checkIngdname(df0.name1[n]))<=2:
                    df0.name1[n] = checkIngdname(df0.name1[n])[0]
                elif checkIngdname(df0['input'][n])!=['and'] and len(checkIngdname(df0['input'][n]))<=2:
                    df0.name1[n] = checkIngdname(df0['input'][n])[0]

    return df0


In [ ]:
# CheckNFixMissingDescript4IngName
def CheckNFixMissingDescript4IngName(df0):
    
    for n in range(len(df0.name1)): 
        if n !=np.nan:
            
            for w in df0.name1[n]:
                if w in [s for s in range(10)]:
                    raise ValueError
            
            if 'mozzarella' in df0.name1[n]:
                continue

            #elif 
                
            elif ('sundried tomato,' in df0['input'][n])==True and ('sundried tomato,' in df0['name1'][n])==False:
                 df0['name1'][n] = 'sundried tomato'

            elif ('oz' in df0['input'][n])==True and ('oz' in df0['name1'][n])==True: # and pd.isnull(df0.name)==True:            
                 #df0['name1'][n] = ' '.join([i for i in df0.name1[n].split(' ') if 'oz' not in i])
                df0['name1'][n] = np.nan    
                raise ValueError
            
            elif ('lb' in df0['input'][n])==True and ('lb' in df0['name1'][n])==True: # and pd.isnull(df0.name)==True:            
                #df0['name1'][n] = ' '.join([i for i in df0.name1[n].split(' ') if 'lb' not in i])
                df0['name1'][n] = np.nan
                raise ValueError

            elif ('/' in df0['input'][n])==True and ('/' in df0['name1'][n])==True: # and pd.isnull(df0.name)==True:            
                #df0['name1'][n] = ' '.join([i for i in df0.name1[n].split(' ') if '/' not in i])
                df0['name1'][n] = np.nan
                raise ValueError

            elif ('-' in df0['input'][n])==True and ('-' in df0['name1'][n])==True: # and pd.isnull(df0.name)==True:  
                if len(getdashIndices(df0['name1'][n]))>1:
                #if 'spring-onion' not in df0['name1'][n]:
                    #df0['name1'][n] = ' '.join([i.lower() for i in df0.name1[n].split(' ') if '-' not in i])
                    df0['name1'][n] = np.nan
                    raise ValueError
            
#             # if qty1/unit1 == empty or nan --> check if input contains 'to serve' --> name1
#             if pd.isnull(df0.unit1[n])==True and pd.isnull(df0.name[n])==True: #== np.nan:
#                 if 'to serve' in df0.input[n] :
#                     df0.name1[n] = df0.input[n][:df0.input[n].find('to serve')] + 'to serve'
#                 if 'to dust' in df0.input[n]:
#                     df0.name1[n] = df0.input[n][:df0.input[n].find('to dust')] + 'to dust'
#                 if 'for dusting' in df0.input[n]:
#                     df0.name1[n] = df0.input[n][:df0.input[n].find('for dusting')] + 'for dusting'
#                 if 'for decorating' in df0.input[n]:
#                     df0.name1[n] = df0.input[n][:df0.input[n].find('for decorating')] + 'for decorating'
            
                
    return df0


In [ ]:
# CheckNFixTMI4IngName
def CheckNFixTMI4IngName(df0):
    for n in range(len(df0.name1)):  
        idx = getcommaIndices(df0.name1[n])
        #print(idx)
        if idx!=[]:
            df0.name1[n] = df0.name1[n][:idx[0]]
            
        idx2 = getBracketIndices(df0.name1[n])
        #print(idx2)
        if idx2!=[]:
            df0.name1[n] = df0.name1[n][:idx2[0]]
    
    return df0

In [ ]:
# CheckNFixUnits
def CheckNFixUnits(df0):
    for n in range(len(df0.unit1)):
        if df0.unit1[n] in unitlabels:
            continue
        elif df0.unit1[n] not in unitlabels:
            df0.unit1[n] = np.nan
        if df0.qty1[n] == 'a':
            df0.qty1[n] = np.nan
            
    return df0

### ==========================================================================================

In [ ]:
# RUN fundtions ############################

collectErrorIDX=[]

for d in range(len(DF)):
# for d in ErrorIDX12.idx:
    #print(d)
    try:
        outtest = utils.import_data(open(crf_filespath +'bbcgdfd_'+str(d)+'o.txt')) 

        df0 = pd.DataFrame(outtest).drop('display',axis=1)
        # df0

        # q2[n].split()
        if 'unit' not in df0.columns.tolist():
            df0['unit'] = np.nan
        if 'comment' not in df0.columns.tolist():
            df0['comment'] = np.nan
        # df0

        df0['qty1'] = df0.qty
        df0['unit1'] = df0.unit
        df0['name1'] = df0.name
        # # df0.head()

        o = getUnitsInOther(df0)
        # o

        q = getUnitsInQty(df0)
        # q

        qo = combine_qNo(q,o)
        # qo

        misplacedU, misplacedQnS = getmisplacedUnQnS(qo)
        # misplacedU, misplacedQnS

        df0 = checkNevalQty(misplacedQnS,df0,qo)

        df0 = checksNupdatesUnit(df0)
        # df0

        df0=CheckNChangeRepeatedIngName(df0)
        # df0

        df0 = CheckNFixMissingDescript4IngName(df0)
        # df0


        df0 = CheckNFixTMI4IngName(df0)
        # df0

        df0 = CheckNFixUnits(df0)
        # df0
        
        #filepath = '/Users/hrm/Documents/Dropbox/DSrelated/Metis/recipes/bbc_goodfood_recipes_pickles/'
        savefilename = filepath + 'bbcgdfd_'+str(d)+'_cleaner02.pkl'
        df0.to_pickle(savefilename)
    
    except:
        collectErrorIDX.append(d)
        
len(collectErrorIDX)        

ErrorList = pd.DataFrame(collectErrorIDX)
ErrorList.to_csv('recipeTagErrorList.csv')

In [ ]:
# len(collectErrorIDX) #2841 #30
# collectErrorIDX

### Check & Explore ==========================================================================================

In [2]:
import pandas as pd

In [3]:
filepath = '/Users/hrm/Documents/Dropbox/DSrelated/Metis/recipes/bbc_goodfood_recipes_pickles/'
DF = pd.read_pickle(filepath+'BBCgoodfood_10081recipesNlinks.pkl')

Modelpath = '/Users/hrm/Documents/Dropbox/DSrelated/Metis/recipes/ingredient-phrase-tagger-master/'
Folderpath = '/Users/hrm/Documents/Dropbox/DSrelated/Metis/recipes'

crf_filespath = Folderpath +'/crf_tagger_files/'

filename = [crf_filespath + 'bbcgdfd_'+str(i)+'.txt' for i in range(10081)]

In [4]:
DF.title[150]

'Chunky minestrone soup'

In [5]:
df1 = pd.read_pickle(filepath + 'bbcgdfd_'+str(150)+'_cleaned.pkl')

In [6]:
df1

,comment,input,name,other,qty,unit,qty1,unit1,name1
0,large roughly chopped,"3 large carrot, roughly chopped",carrot,",",3,NaN,3,NaN,carrot
1,large roughly chopped,"1 large onion, roughly chopped",onion,",",1,NaN,1,NaN,onion
2,roughly chopped,"4 celery sticks, roughly chopped",celery sticks,",",4,NaN,4,NaN,celery sticks
3,NaN,1 tablespoon olive oil,olive oil,NaN,1,tablespoon,1,tablespoon,olive oil
4,crushed,"2 garlic clove, crushed",garlic clove,",",2,NaN,2,NaN,garlic clove
5,large cut into small dice,"2 large potato, cut into small dice",potato,",",2,NaN,2,NaN,potato
6,NaN,2 tablespoon tomato purée,tomato purée,NaN,2,tablespoon,2,tablespoon,tomato purée
7,NaN,2liter vegetable stock,vegetable stock,NaN,2liter,NaN,2,liter,vegetable stock
8,chopped,400gram chopped tomato,tomato,NaN,400gram,NaN,400,gram,tomato
9,or cannellini,"400gram butter, or cannellini bean",butter bean,",",400gram,NaN,400,gram,butter bean


In [7]:
DF.title[469]

'Decadent chocolate truffle torte'

In [8]:
df2 = pd.read_pickle(filepath + 'bbcgdfd_'+str(469)+'_cleaned.pkl')

In [9]:
df2

,comment,input,name,other,qty,unit,qty1,unit1,name1
0,NaN,250gram dark-chocolate,dark-chocolate,NaN,250gram,NaN,250,gram,dark-chocolate
1,NaN,2 tablespoon golden syrup,golden syrup,NaN,2,tablespoon,2,tablespoon,golden syrup
2,NaN,568milliliter carton double cream,double cream,NaN,568milliliter,carton,568,milliliter,double cream
3,NaN,4 teaspoon instant coffee granules,instant coffee granules,NaN,4,teaspoon,4,teaspoon,instant coffee granules
4,ground,1 teaspoon ground cinnamon,cinnamon,NaN,1,teaspoon,1,teaspoon,cinnamon
5,for dusting,"cocoa powder, for dusting",cocoa powder,",",NaN,NaN,NaN,NaN,cocoa powder
